In [44]:
import numpy as np
from skimage.io import imread
from skimage.util import pad
from keras.models import load_model
import gdal
from tqdm import tqdm
import tensorflow as tf

In [45]:
def preprocessing_image_rgb(x):
    # define mean and std values
    mean = [87.845, 96.965, 103.947]
    std = [23.657, 16.474, 13.793]
    # loop over image channels
    for idx, mean_value in enumerate(mean):
        x[..., idx] -= mean_value
        x[..., idx] /= std[idx]
    return x


def preprocessing_image_ms(x):
    # define mean and std values
    mean = [1353.036, 1116.468, 1041.475, 945.344, 1198.498, 2004.878,
            2376.699, 2303.738, 732.957, 12.092, 1818.820, 1116.271, 2602.579]
    std = [65.479, 154.008, 187.997, 278.508, 228.122, 356.598, 456.035,
           531.570, 98.947, 1.188, 378.993, 303.851, 503.181]
    # loop over image channels
    for idx, mean_value in enumerate(mean):
        x[..., idx] -= mean_value
        x[..., idx] /= std[idx]
    return x


def categorical_label_from_full_file_name(files, class_indices):
    from keras.utils import to_categorical
    import os
    # file basename without extension
    base_name = [os.path.splitext(os.path.basename(i))[0] for i in files]
    # class label from filename
    base_name = [i.split("_")[0] for i in base_name]
    # label to indices
    image_class = [class_indices[i] for i in base_name]
    # class indices to one-hot-label
    return to_categorical(image_class, num_classes=len(class_indices))


def simple_image_generator(files, class_indices, batch_size=32,
                           rotation_range=0, horizontal_flip=False,
                           vertical_flip=False):
    from skimage.io import imread
    from skimage.transform import rotate
    import numpy as np
    from random import sample, choice

    while True:
        # select batch_size number of samples without replacement
        batch_files = sample(files, batch_size)
        # get one_hot_label
        batch_Y = categorical_label_from_full_file_name(batch_files,
                                                        class_indices)
        # array for images
        batch_X = []
        # loop over images of the current batch
        for idx, input_path in enumerate(batch_files):
            image = np.array(imread(input_path), dtype=float)
            image = preprocessing_image_ms(image)
            # process image
            if horizontal_flip:
                # randomly flip image up/down
                if choice([True, False]):
                    image = np.flipud(image)
            if vertical_flip:
                # randomly flip image left/right
                if choice([True, False]):
                    image = np.fliplr(image)
            # rotate image by random angle between
            # -rotation_range <= angle < rotation_range
            if rotation_range is not 0:
                angle = np.random.uniform(low=-abs(rotation_range),
                                          high=abs(rotation_range))
                image = rotate(image, angle, mode='reflect',
                               order=1, preserve_range=True)
            # put all together
            batch_X += [image]
        # convert lists to np.array
        X = np.array(batch_X)
        Y = np.array(batch_Y)

        yield(X, Y)


In [46]:
path_to_image = "C:/Users/user/harbor16.tif"

In [47]:
path_to_model = "E:/PAPER_RS_DEEPLEARNING/image/RGB/vgg_rgb_transfer_final.06-0.703.hdf5"

In [48]:
path_to_label_image = "E:/PAPER_RS_DEEPLEARNING/image/RGB/harbor16_label.tif"

In [49]:
path_to_prob_image = "E:/PAPER_RS_DEEPLEARNING/image/RGB/harbor16_prob.tif"

In [50]:
image = np.array(imread(path_to_image), dtype=float)

In [51]:
_, num_cols_unpadded, _ = image.shape

In [52]:
model = load_model(path_to_model)

In [53]:
_, input_rows, input_cols, input_channels = model.layers[0].input_shape

In [54]:
_, output_classes = model.layers[-1].output_shape

In [55]:
_, input_rows, input_cols, input_channels, output_classes

(None, 64, 64, 3, 45)

In [56]:
in_rows_half = int(input_rows/2)

In [57]:
in_cols_half = int(input_cols/2)

In [58]:
image = preprocessing_image_rgb(image)

In [59]:
image.shape

(256, 256, 3)

In [60]:
num_rows, num_cols, _ = image.shape

In [61]:
image_classified_prob = np.zeros((num_rows, num_cols, output_classes))

In [62]:
row_images = np.zeros((num_cols_unpadded, input_rows, input_cols, input_channels))

In [63]:
input_cols, num_cols + input_cols

(64, 320)

In [64]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [65]:
for row in tqdm(range(input_rows, num_rows-input_rows), desc="Processing..."):
    for idx, col in enumerate(range(input_cols, num_cols-input_cols)):
        row_images[idx, ... ] = image[row-in_rows_half:row+in_rows_half,
                                     col-in_cols_half:col+in_cols_half, :]
    row_classified = model.predict(row_images, batch_size=128, verbose=0)
    image_classified_prob[row, :, : ] = row_classified

Processing...: 100%|█████████████████████████████████████████████████████████████████| 128/128 [00:35<00:00,  3.67it/s]


In [66]:
image_classified_prob = image_classified_prob[input_rows:num_rows-input_rows, 
                                               input_cols:num_cols-input_cols, :] 


In [67]:
row_classified.shape

(256, 45)

In [68]:
image_classified_prob[0,0]

array([3.36917874e-04, 8.12971921e-05, 4.11036599e-04, 5.31207828e-04,
       2.84259440e-03, 6.38214260e-05, 3.00089027e-07, 1.05941668e-01,
       1.45641332e-06, 4.48789634e-03, 3.84237677e-01, 5.79819549e-03,
       2.94220672e-06, 5.78468243e-05, 7.33589550e-05, 9.17406869e-05,
       1.52382324e-03, 2.07820073e-01, 4.48145962e-04, 6.47111847e-06,
       2.89743650e-03, 2.10048573e-04, 4.10645316e-06, 4.87991609e-04,
       2.43038740e-02, 2.99552516e-06, 2.72829175e-06, 9.50503871e-02,
       2.63555936e-04, 2.92998593e-05, 2.84127571e-04, 1.58397302e-06,
       7.12058900e-05, 3.51332856e-05, 3.84150655e-04, 3.11665563e-03,
       9.85642895e-02, 4.90578823e-05, 1.40921402e-04, 3.17259803e-02,
       1.64499562e-02, 4.36643255e-04, 2.30386081e-06, 1.04453750e-02,
       2.81665183e-04])

In [69]:
image_classified_label = np.argmax(image_classified_prob, axis=-1) 
image_classified_prob = np.sort(image_classified_prob, axis=-1)[..., -1] 


In [70]:
image = gdal.Open(path_to_image, gdal.GA_ReadOnly)

In [71]:
geotransform = image.GetGeoTransform()

In [72]:
driver = gdal.GetDriverByName('GTiff')

In [73]:
file = driver.Create(path_to_label_image, 
                      image_classified_label.shape[1], 
                      image_classified_label.shape[0], 
                      1, 
                      gdal.GDT_Byte, 
                      ['TFW=YES', 'NUM_THREADS=1']) 

# write label file 



In [74]:
file.SetGeoTransform(geotransform) 

0

In [75]:
file.SetProjection(image.GetProjection()) 

0

In [76]:
file.GetRasterBand(1).WriteArray(image_classified_label)

0

In [77]:
file = None 

In [78]:
file = driver.Create(path_to_prob_image, 
                      image_classified_prob.shape[1], 
                      image_classified_prob.shape[0], 
                      1, 
                      gdal.GDT_Float32, 
                      ['TFW=YES', 'NUM_THREADS=1']) 

In [79]:
file.SetGeoTransform(geotransform) 

0

In [80]:
file.SetProjection(image.GetProjection()) 
# write label file 


0

In [81]:
file.GetRasterBand(1).WriteArray(image_classified_prob) 

0

In [82]:
file = None 
image = None 

In [85]:
import glob
from osgeo import gdal
import numpy as np
import os



In [86]:

def getMeanStd(path, n_bands=3, n_max=-1):
    """Get mean and standard deviation from images.

    Parameters
    ----------
    path : str
        Path to training images
    n_bands : int
        Number of spectral bands (3 for RGB, 13 for Sentinel-2)
    n_max : int
        Maximum number of iterations (-1 = all)

    Return
    ------

    """
    if not os.path.isdir(path):
        print("Error: Directory does not exist.")
        return 0
    
    mean_array = [[] for _ in range(n_bands)]
    std_array = [[] for _ in range(n_bands)]

    # iterate over the images
    i = 0
    for tif in glob.glob(path+"harbor16.tif"):
        if (i < n_max) or (n_max == -1):
            ds = gdal.Open(tif)
            for band in range(n_bands):
                mean_array[band].append(
                    np.mean(ds.GetRasterBand(band+1).ReadAsArray()))
                std_array[band].append(
                    np.std(ds.GetRasterBand(band+1).ReadAsArray()))
            i+=1
        else:
            break

    # results
    res_mean = [np.mean(mean_array[band]) for band in range(n_bands)]
    res_std = [np.mean(std_array[band]) for band in range(n_bands)]

    # print results table
    print("Band |   Mean   |   Std")
    print("-"*28)
    for band in range(n_bands):
        print("{band:4d} | {mean:8.3f} | {std:8.3f}".format(
            band=band, mean=res_mean[band], std=res_std[band]))
    
    return res_mean, res_std



In [87]:
if __name__ == "__main__":
    getMeanStd(path='C:/Users/user/', n_bands=3)

Band |   Mean   |   Std
----------------------------
   0 |   97.323 |   80.164
   1 |  121.246 |   67.701
   2 |  113.442 |   69.944
